<a href="https://colab.research.google.com/github/Fahim-Anin/Machine_Learning_Internship/blob/main/Chatbot_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

**Initializer Token, Stem and Bag of Words for Dataset**

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import nltk

nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    pattern_sentence = ["hello", "how", "are", "you"]
    all_words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word(where the length of the array is the same as the number of words in the given list words)
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

a= "How are you?"
a=tokenize(a)
b=[stem(i) for i in a]
print(b)

['how', 'are', 'you', '?']


In [ ]:
import requests
from pathlib import Path
import json
# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("intents.json").is_file():
  print("intents.json already exists, skipping download")
else:
  print("Downloading intents.json")
  request = requests.get("https://raw.githubusercontent.com/Fahim-Anin/Machine_Learning_Internship/main/intents.json")
  with open("intents.json", "wb") as f:
    f.write(request.content)


intents.json already exists, skipping download


In [ ]:
with open("intents.json", "r") as f:
    intents_data = json.load(f)

tags = []
patterns = []

for intent in intents_data["intents"]:
    tags.append(intent["tag"])
    patterns.extend(intent["patterns"])

# Now, 'tags' contains the tags, and 'patterns' contains all patterns
print("Tags:", tags)
print("Patterns:", patterns)

Tags: ['greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
Patterns: ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", "Thank's a lot!", 'Which items do you have?', 'What kinds of items are there?', 'What do you sell?', 'Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?', 'How long does delivery take?', 'How long does shipping take?', 'When do I get my delivery?', 'Tell me a joke!', 'Tell me something funny!', 'Do you know a joke?']


In [ ]:


all_words = []
tags = []
xy = [] #for differentiate pattern wth specific tags
# loop through each sentence in our intents patterns
for intent in intents_data['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]

# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print("Tags:",tags)
print("lowes+stem:", all_words)
print("Patterns+tags:", xy)


Tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
lowes+stem: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
Patterns+tags: [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'), (['Which', 'items', 'do', 'you', 'have', '?'], 'items

In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

54 7


**loading the dataset**

In [ ]:
from torch.utils.data import Dataset, DataLoader
#ChatDataset initializes the information
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()  #it now holds the information about dataset(features of each index, labels, length of samples)
#train_loader is now a PyTorch DataLoader object that  can be used to iterate over batches of data during training.

# During loop through train_loader, it automatically calls the __getitem__ method of ChatDataset to fetch batches of features and labels.
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Train the Model**

In [ ]:

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x= self.l3(x)

        return x
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 1.3245
Epoch [200/1000], Loss: 0.2052
Epoch [300/1000], Loss: 0.0325
Epoch [400/1000], Loss: 0.0095
Epoch [500/1000], Loss: 0.0100
Epoch [600/1000], Loss: 0.0034
Epoch [700/1000], Loss: 0.0012
Epoch [800/1000], Loss: 0.0026
Epoch [900/1000], Loss: 0.0006
Epoch [1000/1000], Loss: 0.0007
final loss: 0.0007


In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [ ]:
import random
FILE = "data.pth"
data = torch.load(FILE)
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Fahim"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)

    #_the maximum values index are assigned in the predicted
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probability = torch.softmax(output, dim=1)
    prob = probability[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents_data['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
Fahim: Hey :-)
Fahim: We sell coffee and tea
Fahim: I do not understand...
Fahim: We accept VISA, Mastercard and Paypal
Fahim: See you later, thanks for visiting
